## Setup

In [2]:
import sys
sys.path.append('../')
del sys
%reload_ext autoreload
%autoreload 2

In [3]:
from modeling.pipeline import Pipeline

## Display functions

In [12]:
def show_pipeline(pipeline):
    for inputs, targets in pipeline.train_loader[:10]:
        print("\nInputs:")
        for key, item in inputs.items():
            print(key, ':', item)

        print("\nTargets:")
        print(targets)

        if sum(targets) > 0:
            print("Good answers:")
            for i in range(len(targets)):
                if targets[i]:
                    print(inputs['choices'][i])
                    
def show_model(pipeline, model):
    for inputs, targets in pipeline.train_loader[:100]:
        features = model.features(inputs)
        outputs = model.pred(features)
        ranks = model.rank(outputs)
        score = model.score(ranks, targets)

        if sum(targets) > 0:
            mask = targets > 0
            grades = outputs[:, -1][mask]
            ranks = ranks[mask]
            print("score: %.3f" % score)
            print("ranks of the true labels:", ranks, ", grades of the true labels:", grades)
            print()

## Data pipeline parameters

In [5]:
batch_size = 64
drop_last = False
test_proportion = 0.2
valid_proportion = 0.2
use_k_fold = False
k_k_fold = None

## Load the data

In [6]:
pipeline = Pipeline(use_k_fold=use_k_fold)
pipeline.process_data(batch_size=batch_size,
                      drop_last=drop_last,
                      test_proportion=test_proportion,
                      valid_proportion=valid_proportion,
                      k=k_k_fold)

Processing the modeling task...
Computing the annotated queries...
Initial length of queries: 0
Object loaded from ../results/../results/task_annotation/v2_0/task/queries_size10k_shuffle_articles1_queries1_seed0.pkl
Object loaded from ../results/../results/task_annotation/v2_1/task/queries_sizemax_shuffle_articles1_queries1_seed0.pkl
Object loaded from ../results/../results/task_annotation/v2_2/task/queries_sizemax_shuffle_articles1_queries1_seed0.pkl
Final length of queries: 61056
Done (elapsed time: 1s).

Computing the annotations...
Initial length of annotations: 0
Object loaded from ../results/task_annotation/v2_0/results/batch_00_complete.csv
   Correcting "n this article, Nevada and Ohio are discussed. The two American states..." to " The two American states..."
   Correcting "In this article, California and Oregon are discussed. The two neighboring states..." to " The two neighboring states..."
   Correcting "In this article, California and Oregon are discussed. The two West Coa

## Oberseve the data

In [7]:
show_pipeline(pipeline)


Inputs:
entities_type_ : person
entities : ['Franco Alfano', 'Placido Domingo']
summaries : ["Franco Alfano was an Italian composer and pianist, best known today for his opera Risurrezione and above all for having completed Puccini's opera Turandot in 1926. He had considerable success with several of his own works during his lifetime.", 'José Plácido Domingo Embil  is a Spanish opera singer, conductor and arts administrator. He has recorded over a hundred complete operas and is well known for his versatility, regularly performing in Italian, French, German, Spanish, English and Russian in the most prestigious opera houses in the world. Although primarily a lirico-spinto tenor for most of his career, especially popular for his Cavaradossi, Hoffmann, Don José, and Canio, he quickly moved into more dramatic roles, becoming the most acclaimed Otello of his generation.']
context : A Big Nose Must Go On WithoutThe Big Star: Yet by the end of Act II, situated below Roxane 's balcony, Mr. Bar

## Score

In [8]:
scores_names = ['average_precision', 'precision_at_k']
n_updates = 100

### Random

In [9]:
from modeling.models import RandomBaseline

model = RandomBaseline(scores_names=scores_names)
pipeline.preview_data(model=model, include_valid=True)

In [10]:
pipeline.evaluate_baseline(model=model, n_updates=n_updates)

Evaluation of the model...




Test Score: 0.07755
Evaluation of the model...




Test Score: 0.01876


In [13]:
show_model(pipeline, model)

AttributeError: 'RandomBaseline' object has no attribute 'score'

### Counts

In [14]:
from modeling.models import CountsBaseline

model = CountsBaseline(scores_names=scores_names)
pipeline.preview_data(model=model, include_valid=True)

Learning answers counts...


In [15]:
pipeline.evaluate_baseline(model=model, n_updates=n_updates)

Evaluation of the model...




Test Score: 0.45010
Evaluation of the model...




Test Score: 0.20096


In [13]:
show_model(pipeline, model)

score: 0.037
ranks of the true labels: tensor([27.]) , grades of the true labels: tensor([0.1000])

score: 1.000
ranks of the true labels: tensor([1.]) , grades of the true labels: tensor([1.])

score: 0.033
ranks of the true labels: tensor([30.]) , grades of the true labels: tensor([0.2500])

score: 0.333
ranks of the true labels: tensor([3.]) , grades of the true labels: tensor([0.2222])

score: 1.000
ranks of the true labels: tensor([1.]) , grades of the true labels: tensor([1.])

score: 0.030
ranks of the true labels: tensor([33.]) , grades of the true labels: tensor([0.1111])

score: 0.700
ranks of the true labels: tensor([ 2.,  1., 30.]) , grades of the true labels: tensor([0.2720, 1.0000, 0.0080])

score: 0.333
ranks of the true labels: tensor([3.]) , grades of the true labels: tensor([0.3077])

score: 0.500
ranks of the true labels: tensor([2.]) , grades of the true labels: tensor([0.2720])

score: 0.021
ranks of the true labels: tensor([48.]) , grades of the true labels: tenso

### Summaries Counts

In [16]:
from modeling.models import SummariesCountBaseline

model = SummariesCountBaseline(scores_names=scores_names)
pipeline.preview_data(model=model, include_valid=True)

In [17]:
scores = pipeline.evaluate_baseline(model=model, n_updates=n_updates)

Evaluation of the model...




Test Score: 0.17984
Evaluation of the model...




Test Score: 0.06781


In [16]:
show_model(pipeline, model)

score: 0.043
ranks of the true labels: tensor([23.]) , grades of the true labels: tensor([0.])

score: 0.020
ranks of the true labels: tensor([50.]) , grades of the true labels: tensor([0.])

score: 0.091
ranks of the true labels: tensor([11.]) , grades of the true labels: tensor([0.2500])

score: 0.111
ranks of the true labels: tensor([9.]) , grades of the true labels: tensor([0.])

score: 0.030
ranks of the true labels: tensor([33.]) , grades of the true labels: tensor([0.])

score: 1.000
ranks of the true labels: tensor([1.]) , grades of the true labels: tensor([1.])

score: 0.065
ranks of the true labels: tensor([39., 17., 34.]) , grades of the true labels: tensor([0., 0., 0.])

score: 0.250
ranks of the true labels: tensor([4.]) , grades of the true labels: tensor([1.])

score: 0.059
ranks of the true labels: tensor([17.]) , grades of the true labels: tensor([0.1000])

score: 0.250
ranks of the true labels: tensor([4.]) , grades of the true labels: tensor([0.3684])

score: 0.017
r

### Summaries Overlap

In [18]:
from modeling.models import SummariesOverlapBaseline

model = SummariesOverlapBaseline(scores_names=scores_names)
pipeline.preview_data(model=model, include_valid=True)

In [19]:
scores = pipeline.evaluate_baseline(model=model, n_updates=n_updates)

Evaluation of the model...




Test Score: 0.15119
Evaluation of the model...




Test Score: 0.04198


In [19]:
show_model(pipeline, model)

score: 0.043
ranks of the true labels: tensor([23.]) , grades of the true labels: tensor([0.])

score: 0.020
ranks of the true labels: tensor([50.]) , grades of the true labels: tensor([0.])

score: 0.125
ranks of the true labels: tensor([8.]) , grades of the true labels: tensor([0.5000])

score: 0.111
ranks of the true labels: tensor([9.]) , grades of the true labels: tensor([0.])

score: 0.030
ranks of the true labels: tensor([33.]) , grades of the true labels: tensor([0.])

score: 1.000
ranks of the true labels: tensor([1.]) , grades of the true labels: tensor([1.])

score: 0.064
ranks of the true labels: tensor([39., 18., 33.]) , grades of the true labels: tensor([0., 0., 0.])

score: 0.250
ranks of the true labels: tensor([4.]) , grades of the true labels: tensor([1.])

score: 0.250
ranks of the true labels: tensor([4.]) , grades of the true labels: tensor([0.3333])

score: 0.032
ranks of the true labels: tensor([31.]) , grades of the true labels: tensor([0.2500])

score: 0.017
ra